In [1]:
import torch
import torch.nn as nn
import onnxruntime as ort


In [2]:
from models.video_to_roll import resnet18

model_path = "./data/model/video_to_roll_best_f1.pth"

model = resnet18()
model.load_state_dict(torch.load(model_path, map_location='cpu'))
model.eval()

ResNet(
  (conv1): Conv2d(5, 64, kernel_size=(11, 11), stride=(2, 2), padding=(4, 4), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)

In [4]:
latest_opset = max(int(k[14:]) for k in vars(torch.onnx) if 'symbolic_opset' in k) - 1

In [4]:
latest_opset

17

In [3]:
output_path = model_path.replace('pth', 'onnx')

In [4]:
output_path

'./data/model/video_to_roll_best_f1.onnx'

In [5]:

batch_size = 1
x = torch.randn(batch_size, 5, 100, 900, requires_grad=True)

torch.onnx.export(
    model,
    x,
    f=output_path,
    export_params=True,
    verbose=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input' : {0 : 'batch_size'},
        'output' : {0 : 'batch_size'}
    }
)